## 1. Imports & configuration

In [2]:
# --- CELLULE 1 : IMPORTS ET CONFIGURATION ---
import os
import logging
from datetime import datetime
import numpy as np
import matplotlib
matplotlib.use('Agg')  # Pour éviter les erreurs tkinter sous Jupyter
import matplotlib.pyplot as plt

from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, count, when, desc, avg, min, udf, percent_rank, countDistinct, stddev, lit, create_map, coalesce, expr, collect_list
)
from pyspark.sql.types import (StringType, FloatType, LongType, DoubleType, TimestampType, StructField, StructType)
from pyspark.ml.feature import (
    VectorAssembler, StandardScaler, StringIndexer, Bucketizer, Tokenizer, HashingTF, IDF
)
from pyspark.ml.clustering import KMeans
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import ClusteringEvaluator, RegressionEvaluator
from pyspark.ml import Pipeline
from pyspark.sql.window import Window
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Configuration du logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("modelisation.log"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

## 2. Variables globales & création des dossiers

In [3]:
# --- CELLULE 2 : VARIABLES GLOBALES ET DOSSIERS ---
DATA_DIR = "./data/processed/parquet"
MODELS_DIR = "./models"
RESULTS_DIR = "./results"
CHECKPOINTS_DIR = "./data/checkpoints"

os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)
os.makedirs(CHECKPOINTS_DIR, exist_ok=True)

## 3. Classe utilitaire MemoryManager

In [4]:
# --- CELLULE 3 : CLASSE MEMORYMANAGER ---
class MemoryManager:
    """Gestionnaire de mémoire pour les DataFrames Spark"""
    def __init__(self):
        self.cached_dfs = []
    def cache_df(self, df, name="unnamed"):
        df.cache()
        self.cached_dfs.append((df, name))
        logger.info(f"DataFrame '{name}' mis en cache")
        return df
    def unpersist_all(self):
        for df, name in self.cached_dfs:
            try:
                df.unpersist()
                logger.info(f"DataFrame '{name}' libéré du cache")
            except Exception as e:
                logger.warning(f"Erreur lors de la libération de '{name}': {e}")
        self.cached_dfs.clear()
    def __enter__(self):
        return self
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.unpersist_all()

## 4. Fonctions Spark & prétraitement

In [5]:
# --- CELLULE 4 : FONCTIONS SPARK & PRÉTRAITEMENT ---
def create_spark_session():
    return SparkSession.builder \
        .appName("Enhanced E-commerce Analytics") \
        .config("spark.driver.memory", "8g") \
        .config("spark.executor.memory", "8g") \
        .config("spark.memory.fraction", "0.8") \
        .config("spark.memory.storageFraction", "0.3") \
        .config("spark.executor.memoryOverhead", "1g") \
        .config("spark.sql.session.timeZone", "UTC") \
        .config("spark.sql.shuffle.partitions", "24") \
        .config("spark.default.parallelism", "8") \
        .config("spark.sql.adaptive.enabled", "true") \
        .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
        .config("spark.sql.adaptive.localShuffleReader.enabled", "true") \
        .config("spark.sql.adaptive.skewJoin.enabled", "true") \
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
        .config("spark.sql.execution.arrow.pyspark.enabled", "false") \
        .config("spark.python.worker.reuse", "true") \
        .config("spark.python.worker.memory", "2g") \
        .config("spark.network.timeout", "800s") \
        .config("spark.locality.wait", "10s") \
        .config("spark.sql.execution.arrow.maxRecordsPerBatch", "1000") \
        .config("spark.sql.streaming.checkpointLocation", CHECKPOINTS_DIR) \
        .config("spark.sql.streaming.stateStore.providerClass", 
                "org.apache.spark.sql.execution.streaming.state.HDFSBackedStateStoreProvider") \
        .config("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true") \
        .config("spark.sql.streaming.stopGracefullyOnShutdown", "true") \
        .master("local[8]") \
        .getOrCreate()

def find_latest_file(directory, pattern):
    files = [f for f in os.listdir(directory) if pattern in f]
    if not files:
        return None
    latest_file = None
    latest_time = 0
    for file in files:
        file_path = os.path.join(directory, file)
        file_time = os.path.getmtime(file_path)
        if file_time > latest_time:
            latest_time = file_time
            latest_file = file
    return latest_file

## 5. Chargement des données

In [6]:
def find_latest_file(directory, pattern):
    """Trouve le fichier le plus récent dans le répertoire correspondant au pattern"""
    files = [f for f in os.listdir(directory) if pattern in f]
    if not files:
        return None
    
    latest_file = None
    latest_time = 0
    
    for file in files:
        file_path = os.path.join(directory, file)
        file_time = os.path.getmtime(file_path)
        if file_time > latest_time:
            latest_time = file_time
            latest_file = file
            
    return latest_file

## 6. Segmentation RFM

In [7]:
def prepare_rfm_segmentation(user_df):
    """Prépare les données pour la segmentation RFM avec améliorations"""
    logger.info("Préparation des données pour segmentation RFM")
    
    # Filtrer les utilisateurs avec au moins une interaction et mettre en cache
    rfm_df = user_df.filter(col("nb_events") > 0).cache()
    
    # Convertir les valeurs manquantes/nulles en 0 pour les métriques RFM
    rfm_df = rfm_df.fillna({
        "recency": 30,  # Valeur max si jamais vu
        "frequency": 0,  # Pas d'achats
        "monetary": 0    # Pas de dépenses
    })
    
    # Créer des buckets pour les métriques RFM - version simplifiée pour éviter les erreurs
    # Utilisation de quantiles fixes pour plus de stabilité
    
    # Récence (inversée: plus petit = meilleur)
    recency_splits = [0, 10, 20, 30, float('inf')]
    
    # Fréquence et Monétaire - calcul dynamique sécurisé
    freq_stats = rfm_df.select(
        avg("frequency").alias("avg_freq"),
        stddev("frequency").alias("std_freq")
    ).collect()[0]
    
    # Si pas de variance, utiliser des splits fixes
    if freq_stats["std_freq"] is None or freq_stats["std_freq"] == 0:
        frequency_splits = [0, 0.5, 1.5, 2.5, float('inf')]
    else:
        avg_freq = freq_stats["avg_freq"] or 0
        frequency_splits = [0, avg_freq * 0.5, avg_freq, avg_freq * 1.5, float('inf')]
    
    # Même logique pour monetary
    monetary_stats = rfm_df.select(
        avg("monetary").alias("avg_monetary"),
        stddev("monetary").alias("std_monetary")
    ).collect()[0]
    
    if monetary_stats["std_monetary"] is None or monetary_stats["std_monetary"] == 0:
        monetary_splits = [0, 100, 500, 1000, float('inf')]
    else:
        avg_monetary = monetary_stats["avg_monetary"] or 0
        monetary_splits = [0, avg_monetary * 0.5, avg_monetary, avg_monetary * 1.5, float('inf')]
    
    logger.info(f"Splits récence: {recency_splits}")
    logger.info(f"Splits fréquence: {frequency_splits}")
    logger.info(f"Splits monétaire: {monetary_splits}")
    
    # Création des bucketizers
    recency_bucketizer = Bucketizer(
        splits=recency_splits, 
        inputCol="recency", 
        outputCol="recency_score",
        handleInvalid="keep"
    )
    
    frequency_bucketizer = Bucketizer(
        splits=frequency_splits, 
        inputCol="frequency", 
        outputCol="frequency_score",
        handleInvalid="keep"
    )
    
    monetary_bucketizer = Bucketizer(
        splits=monetary_splits, 
        inputCol="monetary", 
        outputCol="monetary_score",
        handleInvalid="keep"
    )
    
    # Appliquer les bucketizers
    rfm_df = recency_bucketizer.transform(rfm_df)
    rfm_df = frequency_bucketizer.transform(rfm_df)
    rfm_df = monetary_bucketizer.transform(rfm_df)
    
    # Inverser le score de récence et normaliser tous les scores
    rfm_df = rfm_df.withColumn("recency_score", 5.0 - col("recency_score"))
    rfm_df = rfm_df.withColumn("recency_score", 
                              when(col("recency_score") < 1, 1)
                              .when(col("recency_score") > 5, 5)
                              .otherwise(col("recency_score")))
    
    # Normaliser frequency et monetary scores
    for score_col in ["frequency_score", "monetary_score"]:
        rfm_df = rfm_df.withColumn(score_col, col(score_col) + 1)
        rfm_df = rfm_df.withColumn(score_col, 
                                  when(col(score_col) > 5, 5)
                                  .otherwise(col(score_col)))
    
    # Calculer le statut actif
    rfm_df = rfm_df.withColumn(
        "is_active",
        when(col("frequency") > 0, 1).otherwise(0)
    )
    
    # Calcul du score RFM global
    rfm_df = rfm_df.withColumn(
        "rfm_score", 
        col("recency_score") * 100 + col("frequency_score") * 10 + col("monetary_score")
    )
    
    # Segmentation RFM robuste
    rfm_df = rfm_df.withColumn(
        "rfm_segment",
        when(col("is_active") == 0, "Inactif")
        .when((col("recency_score") >= 4) & (col("frequency_score") >= 4) & (col("monetary_score") >= 4), "Champions")
        .when((col("recency_score") >= 4) & (col("frequency_score") >= 4), "Loyal Customers")
        .when((col("recency_score") >= 3) & (col("monetary_score") >= 4), "Big Spenders")
        .when((col("recency_score") >= 3) & (col("frequency_score") >= 3), "Potential Loyalists")
        .when((col("recency_score") <= 2) & (col("frequency_score") >= 3), "At Risk")
        .when((col("recency_score") <= 2) & (col("frequency_score") <= 2), "Hibernating")
        .when((col("recency_score") >= 4) & (col("frequency_score") <= 2), "New Customers")
        .when((col("recency_score") >= 3) & (col("frequency_score") <= 2), "Need Attention")
        .otherwise("Others")
    )
    
    # Afficher la distribution des segments - utilisation collect() pour éviter les erreurs UDF
    segment_distribution = rfm_df.groupBy("rfm_segment").count().orderBy(desc("count")).collect()
    
    logger.info("Distribution des segments RFM:")
    for row in segment_distribution:
        logger.info(f"{row['rfm_segment']}: {row['count']}")
    
    return rfm_df

## 7. Clustering comportemental

In [8]:
def prepare_behavioral_clustering(user_df):
    """Prépare les données pour le clustering comportemental avec optimisations"""
    logger.info("Préparation des données pour clustering comportemental")
    
    # Sélection des features comportementales pertinentes
    behavior_features = [
        "nb_events", "nb_views", "nb_carts", "nb_purchases", "nb_removes",
        "avg_price_viewed", "avg_price_purchased", "nb_sessions",
        "conversion_rate", "cart_abandonment", "engagement_days"
    ]
    
    # Filtrer et mettre en cache
    clustering_df = user_df.filter(col("nb_events") >= 2).cache()
    logger.info(f"Utilisateurs avec au moins 2 événements: {clustering_df.count()}")
    
    # Remplacer les valeurs nulles par des zéros
    clustering_df = clustering_df.na.fill({
        feature: 0 for feature in behavior_features
    })
    
    # Assembler les features en vecteurs
    assembler = VectorAssembler(
        inputCols=behavior_features,
        outputCol="features_raw",
        handleInvalid="skip"
    )
    clustering_df = assembler.transform(clustering_df)
    
    # Standardiser les features
    scaler = StandardScaler(
        inputCol="features_raw", 
        outputCol="features",
        withStd=True, 
        withMean=True
    )
    
    # Pipeline pour le preprocessing
    preprocessing_pipeline = Pipeline(stages=[scaler])
    preprocessing_model = preprocessing_pipeline.fit(clustering_df)
    clustering_df = preprocessing_model.transform(clustering_df)
    
    return clustering_df, behavior_features

## 8. Entraînement KMeans et analyse des clusters

In [9]:
def train_kmeans_model(df, feature_col="features", k_values=range(2, 11)):
    """Entraîne et évalue plusieurs modèles K-means avec différentes valeurs de k"""
    logger.info("Entraînement des modèles K-means")
    
    # Liste pour stocker les résultats
    silhouette_scores = []
    models = {}
    
    # Évaluateur pour le clustering
    evaluator = ClusteringEvaluator(
        predictionCol="prediction", 
        featuresCol=feature_col,
        metricName="silhouette"
    )
    
    # Tester différentes valeurs de k
    for k in k_values:
        logger.info(f"Essai avec k={k}")
        
        try:
            # Créer et entraîner le modèle
            kmeans = KMeans(
                k=k, 
                seed=42, 
                featuresCol=feature_col,
                maxIter=20,
                tol=1e-4
            )
            model = kmeans.fit(df)
            
            # Faire des prédictions
            predictions = model.transform(df)
            
            # Évaluer le modèle
            silhouette = evaluator.evaluate(predictions)
            logger.info(f"Silhouette pour k={k}: {silhouette}")
            
            # Stocker les résultats
            silhouette_scores.append(silhouette)
            models[k] = model
            
        except Exception as e:
            logger.error(f"Erreur avec k={k}: {str(e)}")
            silhouette_scores.append(-1)  # Score invalide
            models[k] = None
    
    # Trouver la meilleure valeur de k en utilisant numpy
    silhouette_array = np.array(silhouette_scores)
    valid_indices = silhouette_array > -1  # Filtrer les scores invalides
    
    if not np.any(valid_indices):
        raise ValueError("Aucun modèle valide trouvé")
    
    # Trouver l'indice du meilleur score parmi les valides
    best_idx = np.argmax(silhouette_array[valid_indices])
    # Convertir l'indice local (parmi les valides) en indice global
    global_indices = np.where(valid_indices)[0]
    global_best_idx = global_indices[best_idx]
    
    best_k = list(k_values)[global_best_idx]
    best_score = silhouette_scores[global_best_idx]
    best_model = models[best_k]
    
    logger.info(f"Meilleur modèle: k={best_k} avec silhouette={best_score}")
    
    # Générer les prédictions avec le meilleur modèle
    results = best_model.transform(df)
    
    # Afficher la distribution des clusters
    logger.info("Distribution des clusters:")
    results.groupBy("prediction").count().orderBy("prediction").show()
    
    # Ajout de visualisation des scores silhouette
    try:
        plt.figure(figsize=(10, 6))
        plt.plot(list(k_values), silhouette_scores, 'bo-', linewidth=2, markersize=8)
        plt.xlabel('Nombre de clusters', fontsize=12)
        plt.ylabel('Score Silhouette', fontsize=12)
        plt.title('Optimisation du nombre de clusters - Score Silhouette', fontsize=14)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        
        # Créer le timestamp pour le nom de fichier
        timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")
        plt.savefig(f"{RESULTS_DIR}/silhouette_scores_{timestamp_str}.png", dpi=300, bbox_inches='tight')
        plt.close()
        logger.info("Graphique des scores silhouette sauvegardé")
    except Exception as e:
        logger.warning(f"Erreur lors de la sauvegarde du graphique: {str(e)}")
    
    return best_model, results, best_k, silhouette_scores    

def analyze_clusters(df, cluster_col="prediction", feature_cols=None):
    """Analyse les caractéristiques des clusters avec gestion robuste"""
    logger.info("Analyse des caractéristiques des clusters")
    
    # Calculer les moyennes par cluster
    agg_exprs = [count("*").alias("cluster_size")]
    for col_name in feature_cols:
        agg_exprs.append(avg(col(col_name)).alias(f"avg_{col_name}"))

    cluster_stats = df.groupBy(cluster_col).agg(*agg_exprs).orderBy(cluster_col).collect()
    
    # Afficher les statistiques par cluster via logging
    logger.info("Statistiques par cluster:")
    for row in cluster_stats:
        logger.info(f"Cluster {row[cluster_col]}: {dict(row.asDict())}")
    
    return cluster_stats

## 9. Combinaison des segmentations RFM et comportementale

In [10]:
def combine_segmentations(cluster_df, rfm_df):
    """Combine les segmentations RFM et clustering comportemental - VERSION CORRIGÉE SANS UDF"""
    logger.info("Combinaison des segmentations RFM et clustering")
    
    # Debug : Vérifier les données d'entrée
    logger.info(f"Cluster DF count: {cluster_df.count()}")
    logger.info(f"RFM DF count: {rfm_df.count()}")
    
    # Jointure sécurisée avec repartitioning
    cluster_df = cluster_df.repartition(20, "user_id")
    rfm_df = rfm_df.repartition(20, "user_id").select("user_id", "rfm_segment", "rfm_score")
    
    combined_df = cluster_df.join(rfm_df, on="user_id", how="inner")
    
    # Renommer les colonnes pour plus de clarté
    combined_df = combined_df.withColumnRenamed("prediction", "behavior_cluster")
    
    # SOLUTION: Remplacer l'UDF par une expression CASE WHEN native Spark
    # Créer les étiquettes des clusters avec une expression conditionnelle
    combined_df = combined_df.withColumn(
        "behavior_segment",
        when(col("behavior_cluster") == 0, "Explorateurs Occasionnels")
        .when(col("behavior_cluster") == 1, "Acheteurs Fidèles")
        .when(col("behavior_cluster") == 2, "Visiteurs Fréquents")
        .when(col("behavior_cluster") == 3, "Acheteurs à Fort Panier")
        .when(col("behavior_cluster") == 4, "Visiteurs Uniques")
        .when(col("behavior_cluster") == 5, "Convertisseurs Efficaces")
        .when(col("behavior_cluster") == 6, "Indécis (Abandon Panier)")
        .when(col("behavior_cluster") == 7, "Browsers Passifs")
        .when(col("behavior_cluster") == 8, "Acheteurs Impulsifs")
        .otherwise("Segment Non Défini")
    )
    
    # Afficher les distributions via collect() pour éviter les erreurs
    logger.info("Distribution des segments comportementaux:")
    behavior_distribution = combined_df.groupBy("behavior_segment").count().orderBy(desc("count")).collect()
    for row in behavior_distribution:
        logger.info(f"{row['behavior_segment']}: {row['count']}")
    
    # Analyser l'affinité entre segments
    affinity_results = combined_df.groupBy("behavior_segment", "rfm_segment").count().orderBy(desc("count")).collect()
    logger.info("Top 10 affinités entre segments RFM et comportementaux:")
    for i, row in enumerate(affinity_results[:10]):
        logger.info(f"{i+1}. {row['behavior_segment']} + {row['rfm_segment']}: {row['count']}")
    
    return combined_df

## 10. Fonctions de préparation et d'entraînement pour la recommandation produit (ALS)

In [11]:
def prepare_interaction_data(df):
    """Prépare les données d'interaction pour le système de recommandation"""
    logger.info("Préparation des données d'interaction pour recommandations")
    
    # Créer des ratings implicites basés sur les événements
    rating_weights = {
        'view': 1.0,
        'cart': 2.0,
        'purchase': 5.0,
        'remove_from_cart': -1.0
    }
    
    # Convertir en expression Spark SQL
    rating_expr = " + ".join([
        f"CASE WHEN event_type = '{event}' THEN {weight} ELSE 0 END"
        for event, weight in rating_weights.items()
    ])
    
    interactions = df.groupBy("user_id", "product_id") \
        .agg(
            count("*").alias("interaction_count"),
            expr(f"sum({rating_expr})").alias("rating_score"),
            collect_list("event_type").alias("event_types"),
            avg("price").alias("avg_price")
        ) \
        .filter(col("rating_score") > 0) \
        .withColumn("rating", 
                   when(col("rating_score") > 10, 5.0)
                   .when(col("rating_score") > 5, 4.0)
                   .when(col("rating_score") > 2, 3.0)
                   .when(col("rating_score") > 1, 2.0)
                   .otherwise(1.0))
    
    logger.info(f"Interactions préparées: {interactions.count()} paires user-product")
    return interactions

def train_recommendation_models(interactions_df):
    """Entraîne les modèles de recommandation ALS et content-based"""
    logger.info("Entraînement du modèle ALS pour recommandations collaboratives")
    
    with MemoryManager() as mm:
        # Préparation des données pour ALS
        interactions_cached = mm.cache_df(interactions_df, "interactions")
        
        # Correction : cast des IDs en Long
        from pyspark.sql.types import LongType
        recommendation_df = interactions_cached \
            .withColumn("user_id", col("user_id").cast(LongType())) \
            .withColumn("product_id", col("product_id").cast(LongType()))
        
        # Division train/test
        train_data, test_data = recommendation_df.randomSplit([0.8, 0.2], seed=42)
        train_data = mm.cache_df(train_data, "train_data")
        test_data = mm.cache_df(test_data, "test_data")
        
        # Configuration ALS avec validation croisée
        als = ALS(
            userCol="user_id",
            itemCol="product_id",
            ratingCol="rating",
            coldStartStrategy="drop",
            nonnegative=True,
            seed=42
        )
        
        # Grid search pour optimiser les hyperparamètres
        param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [10, 20, 30]) \
            .addGrid(als.regParam, [0.01, 0.1, 1.0]) \
            .addGrid(als.alpha, [1.0, 10.0]) \
            .build()
        
        evaluator = RegressionEvaluator(
            metricName="rmse",
            labelCol="rating",
            predictionCol="prediction"
        )
        
        crossval = CrossValidator(
            estimator=als,
            estimatorParamMaps=param_grid,
            evaluator=evaluator,
            numFolds=3,
            seed=42
        )
        
        # Entraînement du modèle
        logger.info("Recherche des meilleurs hyperparamètres ALS...")
        cv_model = crossval.fit(train_data)
        best_model = cv_model.bestModel
        
        # Évaluation
        predictions = best_model.transform(test_data)
        rmse = evaluator.evaluate(predictions)
        logger.info(f"RMSE du modèle ALS: {rmse:.4f}")
        
        # Sauvegarde du modèle
        timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")
        model_path = f"{MODELS_DIR}/als_model_{timestamp_str}"
        best_model.save(model_path)
        logger.info(f"Modèle ALS sauvegardé: {model_path}")
        
        return best_model, rmse

def generate_recommendations(als_model, user_id, spark, content_vectors=None, num_recommendations=10):
    """Génère des recommandations pour un utilisateur spécifique"""
    try:
        # Recommandations collaboratives via ALS
        user_df = spark.createDataFrame([(user_id,)], ["user_id"])
        user_recommendations = als_model.recommendForUserSubset(user_df, num_recommendations)
        
        if user_recommendations.count() > 0:
            recs = user_recommendations.select("recommendations").collect()[0]["recommendations"]
            collaborative_recs = [(rec["product_id"], rec["rating"]) for rec in recs]
            logger.info(f"Généré {len(collaborative_recs)} recommandations collaboratives pour l'utilisateur {user_id}")
            return collaborative_recs
        else:
            logger.warning(f"Aucune recommandation collaborative pour l'utilisateur {user_id}")
            return []
    except Exception as e:
        logger.error(f"Erreur lors de la génération de recommandations pour {user_id}: {e}")
        return []


## 11 execution du pipeline d'enrainement des modeles

In [12]:
def main():
    # Initialisation de la session Spark
    spark = create_spark_session()
    spark.sparkContext.setLogLevel("WARN")  # Réduire les logs Spark
    logger.info("Session Spark initialisée")
    
    try:
        # Trouver les fichiers les plus récents
        latest_cleaned = find_latest_file(DATA_DIR, "cleaned_data")
        latest_user_behavior = find_latest_file(DATA_DIR, "user_behavior")
        latest_recommendation = find_latest_file(DATA_DIR, "recommendation_data")
        latest_product = find_latest_file(DATA_DIR, "product_data")
        latest_time_series = find_latest_file(DATA_DIR, "time_series_data")
        
        # Vérifier que les fichiers existent
        if not latest_user_behavior:
            raise FileNotFoundError("Fichier user_behavior non trouvé")
        
        # Chargement des données principales
        logger.info("Chargement des données prétraitées")
        
        user_behavior_df = spark.read.parquet(os.path.join(DATA_DIR, latest_user_behavior))
        logger.info(f"Comportements utilisateurs chargés: {user_behavior_df.count()} lignes")
        
        # Exécuter la segmentation RFM
        rfm_segmentation = prepare_rfm_segmentation(user_behavior_df)
        
        # Exécuter la préparation pour le clustering
        behavior_clustering_df, behavior_features = prepare_behavioral_clustering(user_behavior_df)
        
        # Entraîner le modèle K-means
        kmeans_model, cluster_results, best_k, silhouette_scores = train_kmeans_model(
            behavior_clustering_df, feature_col="features", k_values=range(2, 8)
        )
        
        # Analyser les clusters obtenus
        cluster_stats = analyze_clusters(
            cluster_results.select("user_id", "prediction", *behavior_features),
            cluster_col="prediction", 
            feature_cols=behavior_features
        )
        
        # Sauvegarder le modèle
        timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")
        model_path = f"{MODELS_DIR}/kmeans_behavioral_{best_k}_clusters_{timestamp_str}"
        kmeans_model.save(model_path)
        logger.info(f"Modèle K-means sauvegardé: {model_path}")
        
        # Combiner les segmentations
        user_clusters = cluster_results.select("user_id", "prediction")
        combined_segments = combine_segmentations(user_clusters, rfm_segmentation)
        
        # Sauvegarder les segmentations combinées
        combined_output_path = f"{RESULTS_DIR}/combined_segmentation_{timestamp_str}.parquet"
        combined_segments.write.mode("overwrite").format("parquet").save(combined_output_path)
        logger.info(f"Segmentations combinées sauvegardées: {combined_output_path}")
        
        # Créer un dataframe de profils utilisateurs pour les recommandations
        user_profiles = combined_segments.select(
            "user_id", "behavior_cluster", "behavior_segment", "rfm_segment", "rfm_score"
        )
        
        # Sauvegarder les profils utilisateurs
        profiles_output_path = f"{RESULTS_DIR}/user_profiles_{timestamp_str}.parquet"
        user_profiles.write.mode("overwrite").format("parquet").save(profiles_output_path)
        logger.info(f"Profils utilisateurs sauvegardés: {profiles_output_path}")
        
        # === PARTIE RECOMMANDATION DE PRODUIT ===
        try:
            logger.info("=== DÉMARRAGE DE LA PHASE RECOMMANDATION DE PRODUIT ===")
            # 1. Préparer les interactions utilisateur-produit
            recommendation_df = spark.read.parquet(os.path.join(DATA_DIR, latest_recommendation))
            
            # Correction : cast des IDs en Long
            recommendation_df = recommendation_df \
                .withColumn("user_id", col("user_id").cast(LongType())) \
                .withColumn("product_id", col("product_id").cast(LongType()))
            
            interactions = prepare_interaction_data(recommendation_df)
            
            # 2. Entraîner le modèle ALS (filtrage collaboratif)
            als_model, rmse = train_recommendation_models(interactions)
            logger.info("Modèle ALS entraîné. RMSE: {rmse:.4f}")
            
            # 3. Charger les données produit pour affichage (optionnel)
            if latest_product:
                product_data = spark.read.parquet(os.path.join(DATA_DIR, latest_product))
            else:
                product_data = None
            
            # 4. Générer des recommandations pour quelques utilisateurs (exemple: 5 premiers)
            user_ids = [row["user_id"] for row in user_behavior_df.select("user_id").distinct().limit(5).collect()]
            for user_id in user_ids:
                recommandations = generate_recommendations(als_model, user_id, spark, num_recommendations=5)
                logger.info("Recommandations pour l'utilisateur {user_id}: {recommandations}")
                if product_data and recommandations:
                    rec_ids = [pid for pid, _ in recommandations]
                    produits = product_data.filter(col("product_id").isin(rec_ids)).toPandas()
                    for pid, score in recommandations:
                        details = produits[produits["product_id"] == pid]
                        if not details.empty:
                            logger.info(f"Produit recommandé: {details.iloc[0]['category_code']} - {details.iloc[0]['brand']} | Score: {score:.2f}")
                        else:
                            logger.info(f"Produit ID: {pid} | Score: {score:.2f}")
        except Exception as e:
            logger.error(f"Erreur lors de la phase recommandation: {e}")
            import traceback
            logger.error(traceback.format_exc())
        
        try:
            plt.figure(figsize=(10, 6))
            plt.plot(range(2, 8), silhouette_scores, 'bo-', linewidth=2, markersize=8)
            plt.xlabel('Nombre de clusters', fontsize=12)
            plt.ylabel('Score Silhouette', fontsize=12)
            plt.title('Optimisation du nombre de clusters - Score Silhouette', fontsize=14)
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.savefig(f"{RESULTS_DIR}/silhouette_scores_final_{timestamp_str}.png", dpi=300, bbox_inches='tight')
            plt.close()
            logger.info(f"Graphique final des scores silhouette sauvegardé")
        except Exception as e:
            logger.warning(f"Erreur lors de la sauvegarde du graphique final: {str(e)}")
        
        # Afficher un résumé final
        logger.info("=== RÉSUMÉ DE LA SEGMENTATION ===")
        logger.info(f"Nombre d'utilisateurs total: {user_behavior_df.count()}")
        logger.info(f"Utilisateurs pour clustering: {behavior_clustering_df.count()}")
        logger.info(f"Nombre optimal de clusters: {best_k}")
        
        # Calculer le meilleur score silhouette de manière sécurisée
        valid_scores = [score for score in silhouette_scores if score > -1]
        best_silhouette = np.max(valid_scores) if valid_scores else 0
        logger.info(f"Score silhouette optimal: {best_silhouette:.4f}")
        logger.info(f"Utilisateurs segmentés: {combined_segments.count()}")
        
        # Afficher la distribution finale des segments
        logger.info("Distribution finale des segments RFM:")
        rfm_final_distribution = combined_segments.groupBy("rfm_segment").count().orderBy(desc("count")).collect()
        for row in rfm_final_distribution[:10]:
            logger.info(f"{row['rfm_segment']}: {row['count']}")
        
        logger.info("Distribution finale des segments comportementaux:")
        behavior_final_distribution = combined_segments.groupBy("behavior_segment").count().orderBy(desc("count")).collect()
        for row in behavior_final_distribution[:10]:
            logger.info(f"{row['behavior_segment']}: {row['count']}")
        
    except Exception as e:
        logger.error(f"Erreur lors de l'exécution: {str(e)}")
        import traceback
        logger.error(traceback.format_exc())
        raise
    finally:
        spark.stop()
        logger.info("Session Spark fermée")

if __name__ == "__main__":
    main()

2025-05-24 19:59:18,599 - INFO - Session Spark initialisée
2025-05-24 19:59:18,601 - INFO - Chargement des données prétraitées
2025-05-24 19:59:22,995 - INFO - Comportements utilisateurs chargés: 163024 lignes
2025-05-24 19:59:22,996 - INFO - Préparation des données pour segmentation RFM
2025-05-24 19:59:25,349 - INFO - Splits récence: [0, 10, 20, 30, inf]
2025-05-24 19:59:25,350 - INFO - Splits fréquence: [0, 0.03819069584846403, 0.07638139169692806, 0.1145720875453921, inf]
2025-05-24 19:59:25,353 - INFO - Splits monétaire: [0, 16.671183476052605, 33.34236695210521, 50.013550428157814, inf]
2025-05-24 19:59:27,672 - INFO - Distribution des segments RFM:
2025-05-24 19:59:27,673 - INFO - Inactif: 150572
2025-05-24 19:59:27,674 - INFO - At Risk: 12452
2025-05-24 19:59:27,675 - INFO - Préparation des données pour clustering comportemental
2025-05-24 19:59:28,542 - INFO - Utilisateurs avec au moins 2 événements: 117529
2025-05-24 19:59:29,701 - INFO - Entraînement des modèles K-means
2025

+----------+------+
|prediction| count|
+----------+------+
|         0|107723|
|         1|  9806|
+----------+------+



2025-05-24 19:59:51,796 - INFO - Graphique des scores silhouette sauvegardé
2025-05-24 19:59:51,813 - INFO - Analyse des caractéristiques des clusters
2025-05-24 19:59:52,665 - INFO - Statistiques par cluster:
2025-05-24 19:59:52,666 - INFO - Cluster 0: {'prediction': 0, 'cluster_size': 107723, 'avg_nb_events': 7.845093434085571, 'avg_nb_views': 7.777057824234379, 'avg_nb_carts': 0.04321268438495029, 'avg_nb_purchases': 0.02482292546624212, 'avg_nb_removes': 0.0, 'avg_avg_price_viewed': 320.8869577016758, 'avg_avg_price_purchased': 2.1138300084475916, 'avg_nb_sessions': 1.501072194424589, 'avg_conversion_rate': 0.002730616324765064, 'avg_cart_abandonment': 0.027586185556164114, 'avg_engagement_days': 1.0}
2025-05-24 19:59:52,667 - INFO - Cluster 1: {'prediction': 1, 'cluster_size': 9806, 'avg_nb_events': 11.157250662859473, 'avg_nb_views': 8.6962064042423, 'avg_nb_carts': 1.018050173363247, 'avg_nb_purchases': 1.4429940852539263, 'avg_nb_removes': 0.0, 'avg_avg_price_viewed': 372.51225